Objective
About: This was originally a competition hosted
on kaggle.com consisting of a classification
problem in computer vision.
1. Problem statement: Given the dataset
consisting of driver images in car and
corresponding labels for 10 nos. categories (e.g.
safe driving, texting, talking etc.), your task is to
build a classification model to predict the
category for that image.
2. Dataset link: After creating an account on
Kaggle, go to this competition page
(https://www.kaggle.com/c/state-farmdistracted-driver-detection) and Join Competition.
After joining the competition, you can either
download the data on your local machine or you
can use Kaggle kernel (like jupyter notebook) to
build and train your model (recommended as it
provides you with GPU computing to train your
neural network and no need to download the
dataset locally).
Notes
1. You can resubmit till your code is not evaluated
by TA.
2. This is a concluded competition and therefore,
you need to keep aside a part of training data as
test data to evaluate your model.
3. If you want, you may use Transfer Learning
(building your model on top of pretrained open
sourced model) in this problem.
Comments : Your code must have proper
comments for better understanding.
Score : Score will be given by the TA based on
your submission.
Submission : You have to upload only. ipynb

In [2]:
# First step is to mount the drive

#import necessary libraries
import tensorflow as tf

import os
import shutil
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import call

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.utils import to_categorical,Sequence

import warnings
warnings.filterwarnings('ignore')

In [3]:
PATH = "/content/drive/MyDrive/Colab Notebooks/Coding_ninja_computer_vision/"

In [4]:
print(os.getcwd())
os.chdir(PATH)
print(os.getcwd())

/content
/content/drive/MyDrive/Colab Notebooks/Coding_ninja_computer_vision


In [5]:
# Dictionary: key - folder name, value - Category
class_map = {'c0': 'safe driving',
            'c1': 'texting - right',
            'c2': 'talking on the phone - right',
            'c3': 'texting - left',
            'c4': 'talking on the phone - left'
            }
train_path = 'data/train'   # Train data path
val_path = 'data/validation'       # Validation data path
classes = os.listdir(train_path)
#del classes[0]
#classes.remove('.DS_Store')
# List of directories in train path
print(classes)

['.DS_Store', 'c4', 'c3', 'c2', 'c1', 'c0']


In [6]:
# This is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1/255.) # 0 - 1

# This is a generator that will read pictures found in subfolers of 'train', and generates
# batches of augmented image data on the fly
train_generator = train_datagen.flow_from_directory(directory=train_path,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    target_size=(224,224))

np.unique(train_generator.classes, return_counts=True)# no class imbalance
# Please note that in the presence of class imbalance, class_weights parameter helps long way!

Found 120 images belonging to 5 classes.


(array([0, 1, 2, 3, 4], dtype=int32), array([24, 24, 24, 24, 24]))

In [7]:
# This is the augmentation configuration we will use for validation
val_datagen = ImageDataGenerator(rescale=1/255.)
val_generator = val_datagen.flow_from_directory(directory=val_path,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    shuffle=False,
                                                    target_size=(224,224))

Found 45 images belonging to 5 classes.


In [11]:
# Sequential is a container in keras which is used to stack layers in order
# First layer should include the input data shape. This is mandatory.
# Padding valid implies no padding
# Padding SAME implies enough padding so that output has the same dimensions as input
# Initial layers identify fewer basic features like edges while later layers identify a lot of more abstract features.
# Therefore, no. of filters increase as we go deeper into the network.
# Batchormalization is standard in convolution layers. Improves the convergence time.
# Activation is applied after the Batch normalization
# Dropout could be used for regularization in the fully-connected part of the network

def image_classifier(nb_classes):
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(5, 5), input_shape=(224, 224, 3), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(0.2))

    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.4))

    model.add(Dense(nb_classes, activation='softmax'))

    return(model)


# Cross Entropy is the standard loss function for classification tasks.
# Adam is the most popular optimizer. Convergence is quick. Could be unstable sometimes.
# Learning Rate Schedulers could be used for stabilizing training process
# The 'metrics' mentioned will be computed during run time. So that we can monitor the progress.

model = image_classifier(nb_classes=5)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 220, 220, 32)      2432      
                                                                 
 batch_normalization_3 (Bat  (None, 220, 220, 32)      128       
 chNormalization)                                                
                                                                 
 activation_3 (Activation)   (None, 220, 220, 32)      0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 110, 110, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 106, 106, 64)      51264     
                                                                 
 batch_normalization_4 (Bat  (None, 106, 106, 64)     

In [15]:
# model.fit returns the history of loss and metrics for train and validation datasets.

model.fit_generator(train_generator,
                           validation_data=val_generator,
                           epochs=50) #steps_per_epoch=120/64.0,validation_steps=45/64.0)

Epoch 1/50
2/2 [==============================] - 1s 498ms/step - loss: 0.2044 - accuracy: 0.9250 - val_loss: 2.5318 - val_accuracy: 0.3778
Epoch 2/50
2/2 [==============================] - 1s 506ms/step - loss: 0.1772 - accuracy: 0.9667 - val_loss: 2.5181 - val_accuracy: 0.3778
Epoch 3/50
2/2 [==============================] - 1s 533ms/step - loss: 0.1645 - accuracy: 0.9750 - val_loss: 2.5156 - val_accuracy: 0.3778
Epoch 4/50
2/2 [==============================] - 1s 541ms/step - loss: 0.2549 - accuracy: 0.9083 - val_loss: 2.5415 - val_accuracy: 0.3556
Epoch 5/50
2/2 [==============================] - 1s 618ms/step - loss: 0.1916 - accuracy: 0.9583 - val_loss: 2.5753 - val_accuracy: 0.3556
Epoch 6/50
2/2 [==============================] - 1s 572ms/step - loss: 0.2378 - accuracy: 0.9167 - val_loss: 2.7225 - val_accuracy: 0.3333
Epoch 7/50
2/2 [==============================] - 1s 487ms/step - loss: 0.1685 - accuracy: 0.9500 - val_loss: 2.8145 - val_accuracy: 0.3333
Epoch 8/50
2/2 [====

# Training is coming correct not the validating part, lets do some other approach

In [16]:
# Import ImageDataGenerator for image preprocessing/augmentation
# This basically creates multiple copies of train images by jittering(adding noise).
# This includes rotating, zooming in, flipping, shifting, etc.

datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest') # 'nearest' is kind of algorithm to fill pixel values while transformation

In [17]:
# Import ImageDataGenerator for image preprocessing/augmentation
# This basically creates multiple copies of train images by jittering(adding noise).
# This includes rotating, zooming in, flipping, shifting, etc.
# This is the augmentation configuration we will use for training

train_datagen = ImageDataGenerator(rescale=1/255.,
                                    rotation_range=20,
                                    height_shift_range=0.2,
                                    zoom_range=0.2)

train_generator = train_datagen.flow_from_directory(directory=train_path,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    target_size=(224,224))

Found 120 images belonging to 5 classes.


In [18]:
val_datagen = ImageDataGenerator(rescale=1/255.)
val_generator = train_datagen.flow_from_directory(directory=val_path,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    shuffle=False,
                                                    target_size=(224,224))

Found 45 images belonging to 5 classes.


In [19]:
# Cross Entropy is the standard loss function for classification tasks.
# Adam is the most popular optimizer. Convergence is quick. Could be unstable sometimes.
# Learning Rate Schedulers could be used for stabilizing training process
# The 'metrics' mentioned will be computed during run time. So that we can monitor the progress.

model = image_classifier(nb_classes=5)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [22]:
# model.fit returns the history of loss and metrics for train and validation datasets.

hist0 = model.fit_generator(train_generator,
                           validation_data=val_generator,
                           epochs=50).history()

Epoch 1/50
2/2 [==============================] - 3s 2s/step - loss: 255.2942 - accuracy: 0.2250 - val_loss: 61.8437 - val_accuracy: 0.2000
Epoch 2/50
2/2 [==============================] - 2s 1s/step - loss: 134.8690 - accuracy: 0.2167 - val_loss: 38.9812 - val_accuracy: 0.2000
Epoch 3/50
2/2 [==============================] - 3s 2s/step - loss: 49.2167 - accuracy: 0.2167 - val_loss: 33.7465 - val_accuracy: 0.2000
Epoch 4/50
2/2 [==============================] - 2s 2s/step - loss: 39.6485 - accuracy: 0.2083 - val_loss: 36.7033 - val_accuracy: 0.2889
Epoch 5/50
2/2 [==============================] - 2s 1s/step - loss: 42.5673 - accuracy: 0.2333 - val_loss: 43.0599 - val_accuracy: 0.2222
Epoch 6/50
2/2 [==============================] - 3s 2s/step - loss: 37.4730 - accuracy: 0.2500 - val_loss: 50.3079 - val_accuracy: 0.1778
Epoch 7/50
2/2 [==============================] - 4s 3s/step - loss: 28.0989 - accuracy: 0.1500 - val_loss: 40.7996 - val_accuracy: 0.1778
Epoch 8/50
2/2 [=========

TypeError: 'dict' object is not callable

# Though images are augmented, results are not good
# Lets go for tranfer learning approach

In [23]:
# Get Inception architecture from keras.applications
from tensorflow.keras.applications.inception_v3 import InceptionV3

def inception_tl(nb_classes, freez_wts):

    trained_model = InceptionV3(include_top=False,weights='imagenet')
    x = trained_model.output
    x = GlobalAveragePooling2D()(x)
    pred_inception= Dense(nb_classes,activation='softmax')(x)
    model = Model(inputs=trained_model.input,outputs=pred_inception)

    for layer in trained_model.layers:
        layer.trainable=(1-freez_wts)

    return(model)

In [24]:
model = inception_tl(nb_classes=5, freez_wts=False)
model.summary()

87910968/87910968 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv2d_9 (Conv2D)           (None, None, None, 32)       864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization_9 (Bat  (None, None, None, 32)       96        ['conv2d_9[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 activation_9 (Activation)   

In [25]:

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
hist1 = model.fit_generator(train_generator,
                           validation_data=val_generator,
                           epochs=100,
                           steps_per_epoch=120/60,validation_steps=45/15).history

Epoch 1/100
2/2 [==============================] - ETA: 0s - loss: 1.7600 - accuracy: 0.1917 

2/2 [==============================] - 68s 18s/step - loss: 1.7600 - accuracy: 0.1917 - val_loss: 5.6651 - val_accuracy: 0.2000
Epoch 2/100
2/2 [==============================] - 2s 854ms/step - loss: 1.3401 - accuracy: 0.6250
Epoch 3/100
2/2 [==============================] - 2s 855ms/step - loss: 0.7475 - accuracy: 0.7750
Epoch 4/100
2/2 [==============================] - 3s 1s/step - loss: 0.3627 - accuracy: 0.8500
Epoch 5/100
2/2 [==============================] - 3s 868ms/step - loss: 0.3752 - accuracy: 0.8583
Epoch 6/100
2/2 [==============================] - 2s 1s/step - loss: 0.2044 - accuracy: 0.9333
Epoch 7/100
2/2 [==============================] - 2s 900ms/step - loss: 0.2085 - accuracy: 0.9167
Epoch 8/100
2/2 [==============================] - 2s 1s/step - loss: 0.2689 - accuracy: 0.9500
Epoch 9/100
2/2 [==============================] - 2s 1s/step - loss: 0.1160 - accuracy: 0.9500
Epoch 10/100
2/2 [==============================] - 4s 2s/step - loss: 0.0533 - accuracy: 0.

KeyboardInterrupt: 

In [26]:
val_preds = model.predict_generator(generator=val_generator)

In [27]:
val_preds_class = val_preds.argmax(axis=1)
val_preds_df = pd.DataFrame({'image':val_generator.filenames, 'prediction':val_preds_class})
val_preds_df.head(10)

,image,prediction
0,c0/img_1005.jpg,0
1,c0/img_104.jpg,0
2,c0/img_139.jpg,0
3,c0/img_208.jpg,0
4,c0/img_231.jpg,0
5,c0/img_262.jpg,0
6,c0/img_292.jpg,0
7,c0/img_34.jpg,0
8,c0/img_981.jpg,0
9,c1/img_1045.jpg,3


In [28]:
#Extracting the class number from above image column
y_validation = []
for i in val_preds_df.index.values:
    y_validation.append(int(val_preds_df.image.values[i].split('/')[0][1]))

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_validation, val_preds_class)

0.9111111111111111

In [ ]:
# This seems to be ok , But however on training data model is overfitted, so needs add regularization term
# this can be done by dropout or batch normaliztion